In [ ]:
#... Don't forget to import datasets and libraries

In [ ]:
# Function to plot prevalence rate by gender across flip percentages
def plot_prevalence_rate(mydf, flip_percentages, seeds):
    plt.figure(figsize=(10, 6))
    
    # Initialize arrays to store prevalence rates across seeds
    male_prevalence = np.zeros((len(seeds), len(flip_percentages)))
    female_prevalence = np.zeros((len(seeds), len(flip_percentages)))
    overall_prevalence = np.zeros((len(seeds), len(flip_percentages)))
    
    for seed_idx, seed in enumerate(seeds):
        for flip_idx, flip_pct in enumerate(flip_percentages):
            np.random.seed(seed)
            mydf2 = mydf.copy()
            mydf2['statusflip'] = mydf2['StatusLabel'].copy()
            
            # Apply flipping only to females with 'Not Depressed' status
            mask = (mydf2['Gender'] == 'Female') & (mydf2['statusflip'] == 'Not Depressed')
            if mask.sum() > 0:
                mydf2.loc[mask, 'statusflip'] = np.random.choice(
                    ['Not Depressed', 'Depressed'],
                    size=mask.sum(),
                    p=[1 - flip_pct, flip_pct]
                )
            
            # Convert to binary
            mydf2['statusflipbin'] = mydf2['statusflip'].map({'Not Depressed': 0, 'Depressed': 1})
            
            # Calculate prevalence rates by gender and overall
            male_mask = (mydf2['Gender'] == 'Male')
            female_mask = (mydf2['Gender'] == 'Female')
            
            male_prevalence[seed_idx, flip_idx] = mydf2.loc[male_mask, 'statusflipbin'].mean()
            female_prevalence[seed_idx, flip_idx] = mydf2.loc[female_mask, 'statusflipbin'].mean()
            overall_prevalence[seed_idx, flip_idx] = mydf2['statusflipbin'].mean()
    
    # Calculate mean and SE across seeds
    male_mean = np.mean(male_prevalence, axis=0)
    female_mean = np.mean(female_prevalence, axis=0)
    overall_mean = np.mean(overall_prevalence, axis=0)
    male_se = sem(male_prevalence, axis=0)
    female_se = sem(female_prevalence, axis=0)
    overall_se = sem(overall_prevalence, axis=0)
    
    # Plot prevalence rates
    plt.plot([p*100 for p in flip_percentages], male_mean, 'o-', color='#1f77b4', label='Male', linewidth=2)
    plt.fill_between([p*100 for p in flip_percentages], male_mean - male_se, male_mean + male_se, 
                     color='#1f77b4', alpha=0.2, label='Male ± SE')
    
    plt.plot([p*100 for p in flip_percentages], female_mean, 'o-', color='#ff7f0e', label='Female', linewidth=2)
    plt.fill_between([p*100 for p in flip_percentages], female_mean - female_se, female_mean + female_se, 
                     color='#ff7f0e', alpha=0.2, label='Female ± SE')
    
    plt.plot([p*100 for p in flip_percentages], overall_mean, 'o-', color='#2ca02c', label='Overall', linewidth=2)
    plt.fill_between([p*100 for p in flip_percentages], overall_mean - overall_se, overall_mean + overall_se, 
                     color='#2ca02c', alpha=0.2, label='Overall ± SE')
    
    plt.xlabel('Flip Percentage (%)')
    plt.ylabel('Prevalence Rate')
    plt.title('Prevalence Rate by Gender Across Flip Percentages')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.ylim(0.0, 1.0)
    
    # Save the plot
    plt.tight_layout()
    plt.savefig('prevalence_rate_by_gender.pdf')
    plt.show()
    
    return male_prevalence, female_prevalence, overall_prevalence

# Call the function to generate the plot
male_prev, female_prev, overall_prev = plot_prevalence_rate(mydf, flip_percentages, seeds)